Modelo entrenado


Modelo free eleccion

In [ ]:
import cv2
import mediapipe as mp
import time
import numpy as np
import os

def overlay_image_alpha(img, img_overlay, pos, alpha_mask):
    
    try:
        x, y = int(pos[0]), int(pos[1])
        h, w = img_overlay.shape[:2]
        img_h, img_w = img.shape[:2]


        y_min = y - h // 2
        y_max = y_min + h
        x_min = x - w // 2
        x_max = x_min + w


        y1_roi = max(0, y_min)
        y2_roi = min(img_h, y_max)
        x1_roi = max(0, x_min)
        x2_roi = min(img_w, x_max)
        
        img_cropped = img[y1_roi:y2_roi, x1_roi:x2_roi]
        

        y1_overlay = max(0, -y_min)
        y2_overlay = max(0, y_max - img_h)
        x1_overlay = max(0, -x_min)
        x2_overlay = max(0, x_max - img_w)

        img_overlay_cropped = img_overlay[y1_overlay : h - y2_overlay, x1_overlay : w - x2_overlay]
        alpha_cropped = alpha_mask[y1_overlay : h - y2_overlay, x1_overlay : w - x2_overlay]


        if img_cropped.shape[:2] != img_overlay_cropped.shape[:2]:
            img_overlay_cropped = cv2.resize(img_overlay_cropped, (img_cropped.shape[1], img_cropped.shape[0]))
            alpha_cropped = cv2.resize(alpha_cropped, (img_cropped.shape[1], img_cropped.shape[0]))


        alpha_factor = alpha_cropped[:, :, None] / 255.0
        
        img[y1_roi:y2_roi, x1_roi:x2_roi] = (
            img_cropped * (1 - alpha_factor) + 
            img_overlay_cropped * alpha_factor
        ).astype(np.uint8)
        
        return img
    
    except Exception as e:
        return img 

def rotate_image(image, angle, center=None, scale=1.0):
    (h, w) = image.shape[:2]
    if center is None:
        center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, scale)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_LINEAR)
    return rotated


try:
    hat_img_orig = cv2.imread('gorro.png', cv2.IMREAD_UNCHANGED)
    if hat_img_orig is None:
        raise FileNotFoundError("No se pudo cargar el gorro")
    
    hat_color = hat_img_orig[:, :, :3]  
    hat_alpha = hat_img_orig[:, :, 3]
    print("Gorro cargado.")
except FileNotFoundError as e:
    print(e)
    hat_img_orig = None

cap = cv2.VideoCapture(1)
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1, color=(0, 255, 0))

with mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as face_mesh:

    mouth_open_threshold = 0.04 
    is_mouth_open = False

    start_time = time.time()
    

    animation_frames = []
    animation_current_frame_idx = 0
    animation_last_update_time = 0
    animation_speed = 0.04 
    try:
        folder_path = "animacion_cartas"
        if os.path.exists(folder_path):
            for filename in sorted(os.listdir(folder_path)):
                if filename.endswith(".png"):
                    img_path = os.path.join(folder_path, filename)
                    frame = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
                    if frame is not None:
                        animation_frames.append(frame)
            print(f"Cargados {len(animation_frames)}.")
    except FileNotFoundError as e:
        print(e)

    while cap.isOpened():
        success, image = cap.read()
        if not success:
            continue

        image.flags.writeable = False
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(image_rgb)
        image.flags.writeable = True
        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:

                # Gorro

                if hat_img_orig is not None:

                    img_h, img_w, _ = image.shape
                    
                    p_top = face_landmarks.landmark[10] 
                    p_left_side = face_landmarks.landmark[234] 
                    p_right_side = face_landmarks.landmark[454] 

                    x_top, y_top = int(p_top.x * img_w), int(p_top.y * img_h)
                    x_left, y_left = int(p_left_side.x * img_w), int(p_left_side.y * img_h)
                    x_right, y_right = int(p_right_side.x * img_w), int(p_right_side.y * img_h)

                    hat_desired_width = np.linalg.norm(np.array([x_left, y_left]) - np.array([x_right, y_right]))
                    hat_desired_width *= 2.

                    angle_radians = np.arctan2(y_right - y_left, x_right - x_left)
                    angle_degrees = np.degrees(angle_radians)
                    angle_degrees = -angle_degrees

                    hat_scale_factor = hat_desired_width / hat_img_orig.shape[1]
                    new_hat_width = int(hat_img_orig.shape[1] * hat_scale_factor)
                    new_hat_height = int(hat_img_orig.shape[0] * hat_scale_factor)
                    
                    if new_hat_width > 0 and new_hat_height > 0:
                        resized_hat_color = cv2.resize(hat_color, (new_hat_width, new_hat_height), interpolation=cv2.INTER_AREA)
                        resized_hat_alpha = cv2.resize(hat_alpha, (new_hat_width, new_hat_height), interpolation=cv2.INTER_AREA)

                        rotated_hat_color = rotate_image(resized_hat_color, angle_degrees)
                        rotated_hat_alpha = rotate_image(resized_hat_alpha, angle_degrees)

                        hat_pos_x = x_top
                        hat_pos_y = y_top - int(new_hat_height * 0.4)

                        image = overlay_image_alpha(image, rotated_hat_color, (hat_pos_x, hat_pos_y), rotated_hat_alpha)


                # Labios
                p_lip_top = face_landmarks.landmark[13]
                p_lip_bottom = face_landmarks.landmark[14]


                p_forehead = face_landmarks.landmark[10]
                p_chin = face_landmarks.landmark[152]
                
                face_height = np.linalg.norm(np.array([p_forehead.x, p_forehead.y]) - np.array([p_chin.x, p_chin.y]))
                
                if face_height > 0: 
                    mouth_distance_norm = abs(p_lip_top.y - p_lip_bottom.y) / face_height
                else:
                    mouth_distance_norm = 0
                
                if mouth_distance_norm > mouth_open_threshold:
                    if not is_mouth_open:
                        is_mouth_open = True
                        animation_current_frame_idx = 0
                        animation_last_update_time = time.time()

                    if time.time() - animation_last_update_time > animation_speed:
                        if len(animation_frames) > 0:
                            animation_current_frame_idx = (animation_current_frame_idx + 1) % len(animation_frames)
                        animation_last_update_time = time.time()

                    if len(animation_frames) > 0:
                        current_frame = animation_frames[animation_current_frame_idx]
                        anim_h, anim_w = current_frame.shape[:2]

                        p_mouth_top = face_landmarks.landmark[13]
                        p_mouth_bottom = face_landmarks.landmark[14]
                        
                        cx_mouth = int((p_mouth_top.x + p_mouth_bottom.x) / 2 * img_w)
                        cy_mouth = int((p_mouth_top.y + p_mouth_bottom.y) / 2 * img_h)

                        p_left = face_landmarks.landmark[234]
                        p_right = face_landmarks.landmark[454]
                        face_width_px = np.linalg.norm(
                            np.array([p_left.x * img_w, p_left.y * img_h]) - 
                            np.array([p_right.x * img_w, p_right.y * img_h])
                        )

                        desired_anim_width = int(face_width_px * 0.8) 
                        scale_factor = desired_anim_width / anim_w
                        
                        new_anim_w = int(anim_w * scale_factor)
                        new_anim_h = int(anim_h * scale_factor)


                        if new_anim_w > 0 and new_anim_h > 0:
                            resized_anim = cv2.resize(current_frame, (new_anim_w, new_anim_h), interpolation=cv2.INTER_AREA)
                            
                            anim_color = resized_anim[:, :, :3]
                            anim_alpha = resized_anim[:, :, 3]


                            dy = (p_right.y * img_h) - (p_left.y * img_h)
                            dx = (p_right.x * img_w) - (p_left.x * img_w)
                            angle_degrees = np.degrees(np.arctan2(dy, dx))

                            rotated_anim_color = rotate_image(anim_color, angle_degrees)
                            rotated_anim_alpha = rotate_image(anim_alpha, angle_degrees)


                            offset_y = int(new_anim_h / 2)
                            
                            image = overlay_image_alpha(
                                image, 
                                rotated_anim_color, 
                                (cx_mouth, cy_mouth + offset_y),
                                rotated_anim_alpha
                            )

                else:
                    if is_mouth_open:
                        is_mouth_open = False


        end_time = time.time()
        start_time = end_time
        cv2.imshow('Filtro de Gorro y Boca Animada', image)

        if cv2.waitKey(5) & 0xFF == 27:
            break

cap.release()
cv2.destroyAllWindows()

Gorro cargado.
Cargados 21 frames de animación.
